In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from scipy.stats import uniform

import weakref 

from bayes_opt import BayesianOptimization
#from root_pandas import read_root


import uproot


#To save some memory we will delete unused variables
class TestClass(object): 
    def check(self): 
        print ("object is alive!") 
    def __del__(self): 
        print ("object deleted") 
        
from concurrent.futures import ThreadPoolExecutor
executor = ThreadPoolExecutor(8)

from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, f1_score


from sklearn.preprocessing import StandardScaler

import gc

In [ ]:
df_scaled = uproot.open('/home/shahid/cbmsoft/Cut_optimization/uncut_data/Project/marius/scaled_data_uproot.root:t1').arrays(library='pd')

In [ ]:
# The following columns will be used to predict whether a reconstructed candidate is a lambda particle or not
#cuts = [ 'chi2geo', 'chi2primneg', 'chi2primpos', 'distance', 'ldl']
cuts = [ 'chi2geo', 'chi2primneg', 'chi2primpos', 'chi2topo',
       'cosineneg', 'cosinepos', 'cosinetopo', 'distance', 'l', 'ldl']

x = df_scaled[cuts].to_numpy()

# The MC information is saved in this y variable
y =pd.DataFrame(df_scaled['issignal'], dtype='int8').to_numpy()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=324)

In [ ]:
from DirectRanker import DirectRanker
aa = DirectRanker(num_features = len(x_train[0]), epoch=2, validation_size=0.2)

In [ ]:
aa.fit(x_train,y_train)

In [ ]:
bb = aa.predict_proba(x_test)

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
from helpers import *
from main import *

In [ ]:
from sklearn.model_selection import StratifiedKFold, train_test_split, PredefinedSplit, GridSearchCV

In [ ]:
def weighted_auc(estimator, x, y_true,  w_0=1, w_1=1 ):
    y_pred = estimator.predict_proba(x)
    w = [w_0  if yi == 1 else w_1  for yi in y_true]
    return roc_auc_score(y_true,y_pred,sample_weights=w)

In [ ]:
def auc(estimator, x, y_true):
    return weighted_auc(estimator, x, y_true,  w_0=1, w_1=1)

In [ ]:
scoring = {'auc': auc}

cv_split = StratifiedKFold(n_splits=2, shuffle=True, random_state=32)
refit = 'auc'
parameters = {'hidden_layers_dr':[[32,16],[64,32,16]], 'learning_rate':[0.001, 0.01]}

cv = GridSearchCV(aa,
                  param_grid=parameters,
                  verbose=10,
                  scoring=scoring,
                  cv=cv_split,
                  refit=refit,
                  n_jobs=6,
                  return_train_score=True)
cv.fit(x_train, y_train)
best_estimator = cv.best_estimator_